In [2]:
!pip install -q condacolab
!pip uninstall matplotlib -y
!pip install -q condacolab
!git clone https://github.com/zorzalerrante/aves.git aves_git
!mamba env update --name base --file aves_git/environment-colab.yml
import condacolab
condacolab.install_mambaforge()
!pip install matplotlib

Found existing installation: matplotlib 3.2.2
Uninstalling matplotlib-3.2.2:
  Successfully uninstalled matplotlib-3.2.2
Cloning into 'aves_git'...
remote: Enumerating objects: 1118, done.
remote: Counting objects: 100% (234/234), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 1118 (delta 110), reused 183 (delta 82), pack-reused 884
Receiving objects: 100% (1118/1118), 177.07 MiB | 25.88 MiB/s, done.
Resolving deltas: 100% (458/458), done.
Checking out files: 100% (288/288), done.
/bin/bash: mamba: command not found
⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:30
🔁 Restarting kernel...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.2 MB 14.7 MB/s 
     |████████████████████████████████| 1.1 MB 54.8 MB/s 
     |███████

In [1]:
import numpy as np
import requests
import pandas as pd
from copy import deepcopy
import pandas._testing as tm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import datetime
from google.colab import files
import matplotlib.pyplot as plt
#from aves.features.utils import normalize_rows
import matplotlib as mpl
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from matplotlib.pyplot import plot

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#Datos
incertidumbre= {'chile' : "F019.EPU.IND.91.M",
        'global' : "F019.EPU.IND.90.M", 
       }

In [3]:
url0="https://si3.bcentral.cl/SieteRestWS/SieteRestWS.ashx?user="
user="185305635"
pw="3EhBFkS5En5I"
i="1996-01-01"
f="2022-12-31"

In [4]:
data = []
for key in incertidumbre:
  url = url0 + user + "&pass=" + pw + "&firstdate=" + i + "&lastdate=" + f + "&timeseries=" + incertidumbre[key]+ "&function=GetSeries"
  response = requests.get(url)
  response = response.json()
  response = response["Series"]["Obs"]
  df_data = pd.DataFrame(response)
  df_data['tipo']=key
  data.append(df_data)

In [5]:
data

[    indexDateString   value statusCode   tipo
 0        01-01-1996   76.62         OK  chile
 1        01-02-1996   75.28         OK  chile
 2        01-03-1996   92.33         OK  chile
 3        01-04-1996   63.84         OK  chile
 4        01-05-1996   33.79         OK  chile
 ..              ...     ...        ...    ...
 312      01-01-2022  271.12         OK  chile
 313      01-02-2022  278.32         OK  chile
 314      01-03-2022  299.94         OK  chile
 315      01-04-2022     320         OK  chile
 316      01-05-2022  336.52         OK  chile
 
 [317 rows x 4 columns],     indexDateString   value statusCode    tipo
 0        01-01-1997   77.11         OK  global
 1        01-02-1997   79.52         OK  global
 2        01-03-1997   68.72         OK  global
 3        01-04-1997   74.05         OK  global
 4        01-05-1997   72.41         OK  global
 ..              ...     ...        ...     ...
 300      01-01-2022  232.71         OK  global
 301      01-02-2022   189

In [6]:
df = pd.concat(data, axis=0, ignore_index=True)
df.columns=['Date', 'valor', 'status','Tipo']
df=df[['Date', 'valor','Tipo']]
df['valor'] = df['valor'].apply(pd.to_numeric, errors='coerce')
df['Tipo'] = df['Tipo'].astype("string")
df = df[df['valor'].notna()]
df['valor'] = df['valor'].round(3)
df['month'] = df['Date'].str[3:5]
df['year'] = df['Date'].str[6:10]
df['day'] = df['Date'].str[0:2]
df['Date']= pd.to_datetime(df[["year", "month", "day"]]).dt.strftime('%Y-%m-%d')
df['Date']=pd.to_datetime(df['Date'])
df=df.pivot(index=["Date","year", 'month', 'day'], columns="Tipo", values="valor").reset_index()
df=df.sort_values(by=['Date'])
df.to_csv('incertidumbre.csv') 

In [7]:
df

Tipo,Date,year,month,day,chile,global
0,1996-01-01,1996,01,01,76.62,NaN
1,1996-02-01,1996,02,01,75.28,NaN
2,1996-03-01,1996,03,01,92.33,NaN
3,1996-04-01,1996,04,01,63.84,NaN
4,1996-05-01,1996,05,01,33.79,NaN
...,...,...,...,...,...,...
312,2022-01-01,2022,01,01,271.12,232.71
313,2022-02-01,2022,02,01,278.32,189.30
314,2022-03-01,2022,03,01,299.94,330.66
315,2022-04-01,2022,04,01,320.00,291.63


In [8]:
!pip install -q plotly==4.2.1

     |████████████████████████████████| 7.2 MB 11.7 MB/s 


In [9]:
import plotly.express as px

fig = px.line(df,x="Date", y='global',title='Índice de incertidumbre de política económica Global y Chile')
fig.add_scatter(x=df['Date'], y=df['chile'],mode='lines')
fig.update_yaxes(title_text='Índice')
fig.update_xaxes(title_text='Año')

fig.update_layout(showlegend=False)

fig.show()

###PROYECCIÓN

El gráfico observado nos muestra el índice de incertidumbre político económico tanto a nivel global como en Chile. La incertidumbre global presenta un alza nunca antes vista con el comienzo del brote de Covid-19, de igual manera desde noviembre de 2019 en Chile por causa del estallido social y luego el brote Covid-19.
Utilizaremos la incertidumbre global como referencia para proyectar el índice de incertidumbre en Chile. Se proyecta que en Chile el índice ira en alza nuevamente debido a la inflación que se ha alcanzado en diversos países, se estima que para septiembre en chile el índice pueda sobrepasar los 420. Esto se explica también a nivel país principalmente a que se acabaron las ayudas del gobierno, a esto se suma el incremento del precio de la canasta básica. En el ámbito político nos encontramos en un proceso de posible nueva constitución.
Se proyecta a corto plazo que la cadena de hoteles ubicada en Frutillar debe reestructurar el negocio desde el punto de vista de los costes, acomodando el nuevo nivel de gasto al nuevo nivel de ingreso. Si bien los niveles críticos de la pandemia fueron a la baja, se esperaba que la industria Hotelera tuviera un repunte, sin embargo, en la actualidad solo se aprecia una ralentización, esto se debe al efecto negativo de la inflación. El aumento del índice de incertidumbre político económico en chile influirá en que ciertos viajeros opten por no desplazarse o por hacerlo menos tiempo, como consecuencia el alojamiento acabaría teniendo menos ingresos en un momento de costes elevados.
